In [ ]:
import os
import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms as T
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np


# now, for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
# Configuration options
k_folds = 5
num_epochs = 30
loss_function = nn.CrossEntropyLoss()

# For fold results
results = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}

# Set fixed random number seed
torch.manual_seed(42)

# Replace MNIST dataset with a custom image dataset
image_folder_path = "/kaggle/input/augmented-20x-10x/20x/20x/augmented"  # Change this to the path of your dataset

# Additional transformation
transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.20),
    T.RandomRotation(degrees=(-45, 45)),
    T.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=(-10, 10)),
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # imagenet means
    # T.RandomErasing(p=0.2, value='random')
])

dataset = ImageFolder(root=image_folder_path, transform=transform)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

# Load ResNet model
import timm

model = timm.create_model('vgg16', pretrained=True)

import torch.nn as nn

# Assuming 'model' is your existing model

# Freeze all parameters in the existing model
for param in model.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model.head.fc.in_features

# Modify the fc layer and add more layers for reducing val loss
model.head.fc = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2)
)

# L2 regularization for all linear layers
l2_lambda = 0.01
for layer in model.head.fc.children():
    if isinstance(layer, nn.Linear):
        layer.weight.data = nn.Parameter(layer.weight.data, requires_grad=True)
        layer.bias.data = nn.Parameter(layer.bias.data, requires_grad=True)
        layer.weight.data = layer.weight.data.add(-l2_lambda * layer.weight.data)

model = model.to(device)
print('--------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = SubsetRandomSampler(train_ids)
    test_subsampler = SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = DataLoader(dataset, batch_size=32, sampler=train_subsampler)
    testloader = DataLoader(dataset, batch_size=32, sampler=test_subsampler)

    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

    # Run the training loop for the defined number of epochs
    for epoch in range(0, num_epochs):
        # Print epoch
        print(f'Starting epoch {epoch + 1}')

        # Set current loss value
        current_loss = 0.0

        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Zero the gradients
            optimizer.zero_grad()

            # Perform forward pass
            outputs = model(inputs)

            # Compute loss
            loss = loss_function(outputs, targets)

            # Perform backward pass
            loss.backward()

            # Perform optimization
            optimizer.step()

            # Print statistics
            current_loss += loss.item()
            if i % 500 == 499:
                print('Loss after mini-batch %5d: %.3f' % (i + 1, current_loss / 500))
                current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Evaluation for this fold
    correct, total = 0, 0
    predictions = []
    targets_list = []
    with torch.no_grad():
        # Iterate over the test data and generate predictions
        for i, data in enumerate(testloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Generate outputs
            outputs = model(inputs)

            # Set total and correct
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            # Save predictions and targets for later calculation of precision, recall, f1
            predictions.extend(predicted.cpu().numpy())
            targets_list.extend(targets.cpu().numpy())

        # Calculate precision, recall, f1-score
        precision = precision_score(targets_list, predictions, average='macro')
        recall = recall_score(targets_list, predictions, average='macro')
        f1 = f1_score(targets_list, predictions, average='macro')

        # Print metrics
        print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
        print('Precision for fold %d: %.3f' % (fold, precision))
        print('Recall for fold %d: %.3f' % (fold, recall))
        print('F1-score for fold %d: %.3f' % (fold, f1))
        print('--------------------------------')

        # Save metrics
        results['accuracy'].append(100.0 * (correct / total))
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['f1'].append(f1)

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
for metric, values in results.items():
    avg_metric = np.mean(values)
    std_metric = np.std(values)
    print(f'Average {metric}: {avg_metric:.3f} ± {std_metric:.3f}')

In [ ]:
import os
import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms as T
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np


# now, for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
# Configuration options
k_folds = 5
num_epochs = 30
loss_function = nn.CrossEntropyLoss()

# For fold results
results = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}

# Set fixed random number seed
torch.manual_seed(42)

# Replace MNIST dataset with a custom image dataset
image_folder_path = "/kaggle/input/augmented-20x-10x/20x/20x/augmented"  # Change this to the path of your dataset

# Additional transformation
transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.20),
    T.RandomRotation(degrees=(-45, 45)),
    T.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=(-10, 10)),
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # imagenet means
    # T.RandomErasing(p=0.2, value='random')
])

dataset = ImageFolder(root=image_folder_path, transform=transform)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

# Load ResNet model
import timm

model = timm.create_model('xception71', pretrained=True)

import torch.nn as nn

# Assuming 'model' is your existing model

# Freeze all parameters in the existing model
for param in model.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model.head.fc.in_features

# Modify the fc layer and add more layers for reducing val loss
model.head.fc = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2)
)

# L2 regularization for all linear layers
l2_lambda = 0.01
for layer in model.head.fc.children():
    if isinstance(layer, nn.Linear):
        layer.weight.data = nn.Parameter(layer.weight.data, requires_grad=True)
        layer.bias.data = nn.Parameter(layer.bias.data, requires_grad=True)
        layer.weight.data = layer.weight.data.add(-l2_lambda * layer.weight.data)

model = model.to(device)
print('--------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = SubsetRandomSampler(train_ids)
    test_subsampler = SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = DataLoader(dataset, batch_size=32, sampler=train_subsampler)
    testloader = DataLoader(dataset, batch_size=32, sampler=test_subsampler)

    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

    # Run the training loop for the defined number of epochs
    for epoch in range(0, num_epochs):
        # Print epoch
        print(f'Starting epoch {epoch + 1}')

        # Set current loss value
        current_loss = 0.0

        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Zero the gradients
            optimizer.zero_grad()

            # Perform forward pass
            outputs = model(inputs)

            # Compute loss
            loss = loss_function(outputs, targets)

            # Perform backward pass
            loss.backward()

            # Perform optimization
            optimizer.step()

            # Print statistics
            current_loss += loss.item()
            if i % 500 == 499:
                print('Loss after mini-batch %5d: %.3f' % (i + 1, current_loss / 500))
                current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Evaluation for this fold
    correct, total = 0, 0
    predictions = []
    targets_list = []
    with torch.no_grad():
        # Iterate over the test data and generate predictions
        for i, data in enumerate(testloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Generate outputs
            outputs = model(inputs)

            # Set total and correct
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            # Save predictions and targets for later calculation of precision, recall, f1
            predictions.extend(predicted.cpu().numpy())
            targets_list.extend(targets.cpu().numpy())

        # Calculate precision, recall, f1-score
        precision = precision_score(targets_list, predictions, average='macro')
        recall = recall_score(targets_list, predictions, average='macro')
        f1 = f1_score(targets_list, predictions, average='macro')

        # Print metrics
        print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
        print('Precision for fold %d: %.3f' % (fold, precision))
        print('Recall for fold %d: %.3f' % (fold, recall))
        print('F1-score for fold %d: %.3f' % (fold, f1))
        print('--------------------------------')

        # Save metrics
        results['accuracy'].append(100.0 * (correct / total))
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['f1'].append(f1)

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
for metric, values in results.items():
    avg_metric = np.mean(values)
    std_metric = np.std(values)
    print(f'Average {metric}: {avg_metric:.3f} ± {std_metric:.3f}')

In [ ]:
import os
import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms as T
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np


# now, for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
# Configuration options
k_folds = 5
num_epochs = 30
loss_function = nn.CrossEntropyLoss()

# For fold results
results = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}

# Set fixed random number seed
torch.manual_seed(42)

# Replace MNIST dataset with a custom image dataset
image_folder_path = "/kaggle/input/augmented-20x-10x/20x/20x/augmented"  # Change this to the path of your dataset

# Additional transformation
transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.20),
    T.RandomRotation(degrees=(-45, 45)),
    T.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=(-10, 10)),
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # imagenet means
    # T.RandomErasing(p=0.2, value='random')
])

dataset = ImageFolder(root=image_folder_path, transform=transform)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

# Load ResNet model
import timm

model = timm.create_model('inception_v3', pretrained=True)

import torch.nn as nn

# Assuming 'model' is your existing model

# Freeze all parameters in the existing model
for param in model.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model.fc.in_features

# Modify the fc layer and add more layers for reducing val loss
model.fc = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2)
)

# L2 regularization for all linear layers
l2_lambda = 0.01
for layer in model.fc.children():
    if isinstance(layer, nn.Linear):
        layer.weight.data = nn.Parameter(layer.weight.data, requires_grad=True)
        layer.bias.data = nn.Parameter(layer.bias.data, requires_grad=True)
        layer.weight.data = layer.weight.data.add(-l2_lambda * layer.weight.data)

model = model.to(device)
print('--------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = SubsetRandomSampler(train_ids)
    test_subsampler = SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = DataLoader(dataset, batch_size=32, sampler=train_subsampler)
    testloader = DataLoader(dataset, batch_size=32, sampler=test_subsampler)

    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

    # Run the training loop for the defined number of epochs
    for epoch in range(0, num_epochs):
        # Print epoch
        print(f'Starting epoch {epoch + 1}')

        # Set current loss value
        current_loss = 0.0

        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Zero the gradients
            optimizer.zero_grad()

            # Perform forward pass
            outputs = model(inputs)

            # Compute loss
            loss = loss_function(outputs, targets)

            # Perform backward pass
            loss.backward()

            # Perform optimization
            optimizer.step()

            # Print statistics
            current_loss += loss.item()
            if i % 500 == 499:
                print('Loss after mini-batch %5d: %.3f' % (i + 1, current_loss / 500))
                current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Evaluation for this fold
    correct, total = 0, 0
    predictions = []
    targets_list = []
    with torch.no_grad():
        # Iterate over the test data and generate predictions
        for i, data in enumerate(testloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Generate outputs
            outputs = model(inputs)

            # Set total and correct
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            # Save predictions and targets for later calculation of precision, recall, f1
            predictions.extend(predicted.cpu().numpy())
            targets_list.extend(targets.cpu().numpy())

        # Calculate precision, recall, f1-score
        precision = precision_score(targets_list, predictions, average='macro')
        recall = recall_score(targets_list, predictions, average='macro')
        f1 = f1_score(targets_list, predictions, average='macro')

        # Print metrics
        print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
        print('Precision for fold %d: %.3f' % (fold, precision))
        print('Recall for fold %d: %.3f' % (fold, recall))
        print('F1-score for fold %d: %.3f' % (fold, f1))
        print('--------------------------------')

        # Save metrics
        results['accuracy'].append(100.0 * (correct / total))
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['f1'].append(f1)

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
for metric, values in results.items():
    avg_metric = np.mean(values)
    std_metric = np.std(values)
    print(f'Average {metric}: {avg_metric:.3f} ± {std_metric:.3f}')

In [ ]:
import os
import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms as T
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np


# now, for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
# Configuration options
k_folds = 5
num_epochs = 30
loss_function = nn.CrossEntropyLoss()

# For fold results
results = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}

# Set fixed random number seed
torch.manual_seed(42)

# Replace MNIST dataset with a custom image dataset
image_folder_path = "/kaggle/input/augmented-20x-10x/20x/20x/augmented"  # Change this to the path of your dataset

# Additional transformation
transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.20),
    T.RandomRotation(degrees=(-45, 45)),
    T.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=(-10, 10)),
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # imagenet means
    # T.RandomErasing(p=0.2, value='random')
])

dataset = ImageFolder(root=image_folder_path, transform=transform)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

# Load ResNet model
import timm

model = timm.create_model('inception_v3', pretrained=True)

import torch.nn as nn

# Assuming 'model' is your existing model

# Freeze all parameters in the existing model
for param in model.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model.head.fc.in_features

# Modify the fc layer and add more layers for reducing val loss
model.head = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2)
)

# L2 regularization for all linear layers
l2_lambda = 0.01
for layer in model.head.fc.children():
    if isinstance(layer, nn.Linear):
        layer.weight.data = nn.Parameter(layer.weight.data, requires_grad=True)
        layer.bias.data = nn.Parameter(layer.bias.data, requires_grad=True)
        layer.weight.data = layer.weight.data.add(-l2_lambda * layer.weight.data)

model = model.to(device)
print('--------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = SubsetRandomSampler(train_ids)
    test_subsampler = SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = DataLoader(dataset, batch_size=32, sampler=train_subsampler)
    testloader = DataLoader(dataset, batch_size=32, sampler=test_subsampler)

    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

    # Run the training loop for the defined number of epochs
    for epoch in range(0, num_epochs):
        # Print epoch
        print(f'Starting epoch {epoch + 1}')

        # Set current loss value
        current_loss = 0.0

        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Zero the gradients
            optimizer.zero_grad()

            # Perform forward pass
            outputs = model(inputs)

            # Compute loss
            loss = loss_function(outputs, targets)

            # Perform backward pass
            loss.backward()

            # Perform optimization
            optimizer.step()

            # Print statistics
            current_loss += loss.item()
            if i % 500 == 499:
                print('Loss after mini-batch %5d: %.3f' % (i + 1, current_loss / 500))
                current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Evaluation for this fold
    correct, total = 0, 0
    predictions = []
    targets_list = []
    with torch.no_grad():
        # Iterate over the test data and generate predictions
        for i, data in enumerate(testloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Generate outputs
            outputs = model(inputs)

            # Set total and correct
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            # Save predictions and targets for later calculation of precision, recall, f1
            predictions.extend(predicted.cpu().numpy())
            targets_list.extend(targets.cpu().numpy())

        # Calculate precision, recall, f1-score
        precision = precision_score(targets_list, predictions, average='macro')
        recall = recall_score(targets_list, predictions, average='macro')
        f1 = f1_score(targets_list, predictions, average='macro')

        # Print metrics
        print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
        print('Precision for fold %d: %.3f' % (fold, precision))
        print('Recall for fold %d: %.3f' % (fold, recall))
        print('F1-score for fold %d: %.3f' % (fold, f1))
        print('--------------------------------')

        # Save metrics
        results['accuracy'].append(100.0 * (correct / total))
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['f1'].append(f1)

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
for metric, values in results.items():
    avg_metric = np.mean(values)
    std_metric = np.std(values)
    print(f'Average {metric}: {avg_metric:.3f} ± {std_metric:.3f}')

In [ ]:
import os
import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms as T
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np


# now, for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
# Configuration options
k_folds = 5
num_epochs = 30
loss_function = nn.CrossEntropyLoss()

# For fold results
results = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}

# Set fixed random number seed
torch.manual_seed(42)

# Replace MNIST dataset with a custom image dataset
image_folder_path = "/kaggle/input/augmented-20x-10x/20x/20x/augmented"  # Change this to the path of your dataset

# Additional transformation
transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.20),
    T.RandomRotation(degrees=(-45, 45)),
    T.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=(-10, 10)),
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # imagenet means
    # T.RandomErasing(p=0.2, value='random')
])

dataset = ImageFolder(root=image_folder_path, transform=transform)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

# Load ResNet model
import timm

model = timm.create_model('mobilenetv2_140', pretrained=True)

import torch.nn as nn

# Assuming 'model' is your existing model

# Freeze all parameters in the existing model
for param in model.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model.head.fc.in_features

# Modify the fc layer and add more layers for reducing val loss
model.head = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2)
)

# L2 regularization for all linear layers
l2_lambda = 0.01
for layer in model.head.fc.children():
    if isinstance(layer, nn.Linear):
        layer.weight.data = nn.Parameter(layer.weight.data, requires_grad=True)
        layer.bias.data = nn.Parameter(layer.bias.data, requires_grad=True)
        layer.weight.data = layer.weight.data.add(-l2_lambda * layer.weight.data)

model = model.to(device)
print('--------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = SubsetRandomSampler(train_ids)
    test_subsampler = SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = DataLoader(dataset, batch_size=32, sampler=train_subsampler)
    testloader = DataLoader(dataset, batch_size=32, sampler=test_subsampler)

    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

    # Run the training loop for the defined number of epochs
    for epoch in range(0, num_epochs):
        # Print epoch
        print(f'Starting epoch {epoch + 1}')

        # Set current loss value
        current_loss = 0.0

        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Zero the gradients
            optimizer.zero_grad()

            # Perform forward pass
            outputs = model(inputs)

            # Compute loss
            loss = loss_function(outputs, targets)

            # Perform backward pass
            loss.backward()

            # Perform optimization
            optimizer.step()

            # Print statistics
            current_loss += loss.item()
            if i % 500 == 499:
                print('Loss after mini-batch %5d: %.3f' % (i + 1, current_loss / 500))
                current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Evaluation for this fold
    correct, total = 0, 0
    predictions = []
    targets_list = []
    with torch.no_grad():
        # Iterate over the test data and generate predictions
        for i, data in enumerate(testloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Generate outputs
            outputs = model(inputs)

            # Set total and correct
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            # Save predictions and targets for later calculation of precision, recall, f1
            predictions.extend(predicted.cpu().numpy())
            targets_list.extend(targets.cpu().numpy())

        # Calculate precision, recall, f1-score
        precision = precision_score(targets_list, predictions, average='macro')
        recall = recall_score(targets_list, predictions, average='macro')
        f1 = f1_score(targets_list, predictions, average='macro')

        # Print metrics
        print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
        print('Precision for fold %d: %.3f' % (fold, precision))
        print('Recall for fold %d: %.3f' % (fold, recall))
        print('F1-score for fold %d: %.3f' % (fold, f1))
        print('--------------------------------')

        # Save metrics
        results['accuracy'].append(100.0 * (correct / total))
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['f1'].append(f1)

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
for metric, values in results.items():
    avg_metric = np.mean(values)
    std_metric = np.std(values)
    print(f'Average {metric}: {avg_metric:.3f} ± {std_metric:.3f}')

In [ ]:
import os
import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms as T
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np


# now, for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
# Configuration options
k_folds = 5
num_epochs = 30
loss_function = nn.CrossEntropyLoss()

# For fold results
results = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}

# Set fixed random number seed
torch.manual_seed(42)

# Replace MNIST dataset with a custom image dataset
image_folder_path = "/kaggle/input/augmented-20x-10x/20x/20x/augmented"  # Change this to the path of your dataset

# Additional transformation
transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.20),
    T.RandomRotation(degrees=(-45, 45)),
    T.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=(-10, 10)),
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # imagenet means
    # T.RandomErasing(p=0.2, value='random')
])

dataset = ImageFolder(root=image_folder_path, transform=transform)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

# Load ResNet model
import timm

model = timm.create_model('mobilenetv2_140', pretrained=True)

import torch.nn as nn

# Assuming 'model' is your existing model

# Freeze all parameters in the existing model
for param in model.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model.classifier.in_features

# Modify the fc layer and add more layers for reducing val loss
model.classifier = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2)
)

# L2 regularization for all linear layers
l2_lambda = 0.01
for layer in model.classifier.children():
    if isinstance(layer, nn.Linear):
        layer.weight.data = nn.Parameter(layer.weight.data, requires_grad=True)
        layer.bias.data = nn.Parameter(layer.bias.data, requires_grad=True)
        layer.weight.data = layer.weight.data.add(-l2_lambda * layer.weight.data)

model = model.to(device)
print('--------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = SubsetRandomSampler(train_ids)
    test_subsampler = SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = DataLoader(dataset, batch_size=32, sampler=train_subsampler)
    testloader = DataLoader(dataset, batch_size=32, sampler=test_subsampler)

    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

    # Run the training loop for the defined number of epochs
    for epoch in range(0, num_epochs):
        # Print epoch
        print(f'Starting epoch {epoch + 1}')

        # Set current loss value
        current_loss = 0.0

        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Zero the gradients
            optimizer.zero_grad()

            # Perform forward pass
            outputs = model(inputs)

            # Compute loss
            loss = loss_function(outputs, targets)

            # Perform backward pass
            loss.backward()

            # Perform optimization
            optimizer.step()

            # Print statistics
            current_loss += loss.item()
            if i % 500 == 499:
                print('Loss after mini-batch %5d: %.3f' % (i + 1, current_loss / 500))
                current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Evaluation for this fold
    correct, total = 0, 0
    predictions = []
    targets_list = []
    with torch.no_grad():
        # Iterate over the test data and generate predictions
        for i, data in enumerate(testloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Generate outputs
            outputs = model(inputs)

            # Set total and correct
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            # Save predictions and targets for later calculation of precision, recall, f1
            predictions.extend(predicted.cpu().numpy())
            targets_list.extend(targets.cpu().numpy())

        # Calculate precision, recall, f1-score
        precision = precision_score(targets_list, predictions, average='macro')
        recall = recall_score(targets_list, predictions, average='macro')
        f1 = f1_score(targets_list, predictions, average='macro')

        # Print metrics
        print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
        print('Precision for fold %d: %.3f' % (fold, precision))
        print('Recall for fold %d: %.3f' % (fold, recall))
        print('F1-score for fold %d: %.3f' % (fold, f1))
        print('--------------------------------')

        # Save metrics
        results['accuracy'].append(100.0 * (correct / total))
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['f1'].append(f1)

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
for metric, values in results.items():
    avg_metric = np.mean(values)
    std_metric = np.std(values)
    print(f'Average {metric}: {avg_metric:.3f} ± {std_metric:.3f}')

In [ ]:
import os
import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms as T
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np


# now, for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
# Configuration options
k_folds = 5
num_epochs = 30
loss_function = nn.CrossEntropyLoss()

# For fold results
results = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}

# Set fixed random number seed
torch.manual_seed(42)

# Replace MNIST dataset with a custom image dataset
image_folder_path = "/kaggle/input/augmented-20x-10x/20x/20x/augmented"  # Change this to the path of your dataset

# Additional transformation
transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.20),
    T.RandomRotation(degrees=(-45, 45)),
    T.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=(-10, 10)),
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # imagenet means
    # T.RandomErasing(p=0.2, value='random')
])

dataset = ImageFolder(root=image_folder_path, transform=transform)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

# Load ResNet model
import timm

model = timm.create_model('efficientnet_b4', pretrained=True)

import torch.nn as nn

# Assuming 'model' is your existing model

# Freeze all parameters in the existing model
for param in model.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model.classifier.in_features

# Modify the fc layer and add more layers for reducing val loss
model.classifier = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2)
)

# L2 regularization for all linear layers
l2_lambda = 0.01
for layer in model.classifier.children():
    if isinstance(layer, nn.Linear):
        layer.weight.data = nn.Parameter(layer.weight.data, requires_grad=True)
        layer.bias.data = nn.Parameter(layer.bias.data, requires_grad=True)
        layer.weight.data = layer.weight.data.add(-l2_lambda * layer.weight.data)

model = model.to(device)
print('--------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = SubsetRandomSampler(train_ids)
    test_subsampler = SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = DataLoader(dataset, batch_size=32, sampler=train_subsampler)
    testloader = DataLoader(dataset, batch_size=32, sampler=test_subsampler)

    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

    # Run the training loop for the defined number of epochs
    for epoch in range(0, num_epochs):
        # Print epoch
        print(f'Starting epoch {epoch + 1}')

        # Set current loss value
        current_loss = 0.0

        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Zero the gradients
            optimizer.zero_grad()

            # Perform forward pass
            outputs = model(inputs)

            # Compute loss
            loss = loss_function(outputs, targets)

            # Perform backward pass
            loss.backward()

            # Perform optimization
            optimizer.step()

            # Print statistics
            current_loss += loss.item()
            if i % 500 == 499:
                print('Loss after mini-batch %5d: %.3f' % (i + 1, current_loss / 500))
                current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Evaluation for this fold
    correct, total = 0, 0
    predictions = []
    targets_list = []
    with torch.no_grad():
        # Iterate over the test data and generate predictions
        for i, data in enumerate(testloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Generate outputs
            outputs = model(inputs)

            # Set total and correct
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            # Save predictions and targets for later calculation of precision, recall, f1
            predictions.extend(predicted.cpu().numpy())
            targets_list.extend(targets.cpu().numpy())

        # Calculate precision, recall, f1-score
        precision = precision_score(targets_list, predictions, average='macro')
        recall = recall_score(targets_list, predictions, average='macro')
        f1 = f1_score(targets_list, predictions, average='macro')

        # Print metrics
        print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
        print('Precision for fold %d: %.3f' % (fold, precision))
        print('Recall for fold %d: %.3f' % (fold, recall))
        print('F1-score for fold %d: %.3f' % (fold, f1))
        print('--------------------------------')

        # Save metrics
        results['accuracy'].append(100.0 * (correct / total))
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['f1'].append(f1)

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
for metric, values in results.items():
    avg_metric = np.mean(values)
    std_metric = np.std(values)
    print(f'Average {metric}: {avg_metric:.3f} ± {std_metric:.3f}')

In [ ]:
import os
import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms as T
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np


# now, for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
# Configuration options
k_folds = 5
num_epochs = 30
loss_function = nn.CrossEntropyLoss()

# For fold results
results = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}

# Set fixed random number seed
torch.manual_seed(42)

# Replace MNIST dataset with a custom image dataset
image_folder_path = "/kaggle/input/augmented-20x-10x/20x/20x/augmented"  # Change this to the path of your dataset

# Additional transformation
transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.20),
    T.RandomRotation(degrees=(-45, 45)),
    T.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=(-10, 10)),
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # imagenet means
    # T.RandomErasing(p=0.2, value='random')
])

dataset = ImageFolder(root=image_folder_path, transform=transform)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

# Load ResNet model

import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)

import torch.nn as nn

# Assuming 'model' is your existing model

# Freeze all parameters in the existing model
for param in model.parameters():
    param.requires_grad = False

# Get the number of input features of the existing fc layer
n_inputs = model.fc.in_features

# Modify the fc layer and add more layers for reducing val loss
model.fc = nn.Sequential(
    nn.Linear(n_inputs, 1024),
    nn.BatchNorm1d(1024),  # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),   # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),    # Batch Normalization
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2)
)

# L2 regularization for all linear layers
l2_lambda = 0.01
for layer in model.fc.children():
    if isinstance(layer, nn.Linear):
        layer.weight.data = nn.Parameter(layer.weight.data, requires_grad=True)
        layer.bias.data = nn.Parameter(layer.bias.data, requires_grad=True)
        layer.weight.data = layer.weight.data.add(-l2_lambda * layer.weight.data)

model = model.to(device)
print('--------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = SubsetRandomSampler(train_ids)
    test_subsampler = SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = DataLoader(dataset, batch_size=32, sampler=train_subsampler)
    testloader = DataLoader(dataset, batch_size=32, sampler=test_subsampler)

    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

    # Run the training loop for the defined number of epochs
    for epoch in range(0, num_epochs):
        # Print epoch
        print(f'Starting epoch {epoch + 1}')

        # Set current loss value
        current_loss = 0.0

        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Zero the gradients
            optimizer.zero_grad()

            # Perform forward pass
            outputs = model(inputs)

            # Compute loss
            loss = loss_function(outputs, targets)

            # Perform backward pass
            loss.backward()

            # Perform optimization
            optimizer.step()

            # Print statistics
            current_loss += loss.item()
            if i % 500 == 499:
                print('Loss after mini-batch %5d: %.3f' % (i + 1, current_loss / 500))
                current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Evaluation for this fold
    correct, total = 0, 0
    predictions = []
    targets_list = []
    with torch.no_grad():
        # Iterate over the test data and generate predictions
        for i, data in enumerate(testloader, 0):
            # Get inputs
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            # Generate outputs
            outputs = model(inputs)

            # Set total and correct
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            # Save predictions and targets for later calculation of precision, recall, f1
            predictions.extend(predicted.cpu().numpy())
            targets_list.extend(targets.cpu().numpy())

        # Calculate precision, recall, f1-score
        precision = precision_score(targets_list, predictions, average='macro')
        recall = recall_score(targets_list, predictions, average='macro')
        f1 = f1_score(targets_list, predictions, average='macro')

        # Print metrics
        print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
        print('Precision for fold %d: %.3f' % (fold, precision))
        print('Recall for fold %d: %.3f' % (fold, recall))
        print('F1-score for fold %d: %.3f' % (fold, f1))
        print('--------------------------------')

        # Save metrics
        results['accuracy'].append(100.0 * (correct / total))
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['f1'].append(f1)

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
for metric, values in results.items():
    avg_metric = np.mean(values)
    std_metric = np.std(values)
    print(f'Average {metric}: {avg_metric:.3f} ± {std_metric:.3f}')